# Load DNS and mDNS datasets

In [2]:
import os
import sys

import numpy as np
import pandas as pd

module_path = os.path.abspath(os.path.join('.'))
if module_path not in sys.path:
    sys.path.append(module_path)

import torch
import torch_geometric.transforms as T

from libs.loader import DNS

In [5]:
dns_df = pd.read_csv('./data/DNS/oldlabels.csv')
iplist = pd.read_csv('./data/DNS/nodes.ip.csv')['node_id'].tolist()
iplist
dns_df = dns_df[~dns_df['node_id'].isin(iplist)]
dns_df.to_csv('./data/DNS/extras.csv', index=False)

In [8]:
dns_df = pd.read_csv('./data/DNS/oldlabels.csv')
dns_df = dns_df[~dns_df['node_id'].str.startswith('i')]
dns_df.to_csv('./data/DNS/extras.csv', index=False)
dns_df.tail()

dns_df.groupby(by='type').size().reset_index(name='#count').sort_values(by='#count', ascending=False)

,type,#count
2,n_a,689540
1,malicious,20354
0,benign,4963


In [9]:
mdns_df = pd.read_csv('./data/mDNS/extras.csv')
mdns_df.groupby(by='type').size().reset_index(name='#count').sort_values(by='#count', ascending=False)

,type,#count
0,benign,4668
1,malicious,2827


In [12]:
torch.randperm(5)

tensor([3, 0, 4, 2, 1])

## Load Graphs

In [4]:
kg_path = lambda graph_name: f'./data/{graph_name}'

#### mDNS

In [5]:
dataset = DNS(root=kg_path('mDNS'), transform=T.Compose([T.NormalizeFeatures(), T.ToUndirected()]))
dataset[0]

Remove parallel edges: Series([], dtype: int64)


HeteroData(
  ip_node={
    num_nodes=4505,
    x=[4505, 2]
  },
  domain_node={
    num_nodes=7495,
    x=[7495, 10],
    y=[7495],
    train_mask=[7495],
    test_mask=[7495],
    val_mask=[7495]
  },
  (domain_node, apex, domain_node)={ edge_index=[2, 3320] },
  (ip_node, resolves, domain_node)={ edge_index=[2, 34119] },
  (domain_node, similar, domain_node)={ edge_index=[2, 3012] },
  (domain_node, rev_resolves, ip_node)={ edge_index=[2, 34119] }
)

#### DNS

In [8]:
dataset = DNS(root=kg_path('DNS'), transform=T.Compose([T.NormalizeFeatures(), T.ToUndirected()]))
dataset[0]

Remove parallel edges: type
similar    50910
dtype: int64


HeteroData(
  ip_node={
    num_nodes=73593,
    x=[73593, 2]
  },
  domain_node={
    num_nodes=373475,
    x=[373475, 10],
    y=[447068],
    train_mask=[373475],
    test_mask=[373475],
    val_mask=[373475]
  },
  (domain_node, apex, domain_node)={ edge_index=[2, 178944] },
  (domain_node, resolves, ip_node)={ edge_index=[2, 730438] },
  (domain_node, similar, domain_node)={ edge_index=[2, 155356] },
  (ip_node, rev_resolves, domain_node)={ edge_index=[2, 730438] }
)

In [7]:
dataset.to_homogeneous()

Data(
  node_type=[447068],
  x=[447068, 12],
  edge_index=[2, 897588],
  edge_type=[897588],
  train_mask=[447068],
  val_mask=[447068],
  test_mask=[447068],
  y=[520661],
  edge_map={
    0=[2],
    1=[2],
    2=[2]
  },
  num_nodes=447068
)

In [1]:
import shutil

shutil.make_archive('data', 'zip', './data')

'/home/local/QCRI/fdeniz/ucsc3/data.zip'